In [5]:
import pandas as pd

# Step 1: Load Emails, Text Messages, Voicemails, and Property Info
emails = pd.read_csv('aggregated_email_data.csv')
text_messages = pd.read_csv('updated_text_message_logs.csv')
voicemails = pd.read_csv('updated_voicemail_logs.csv')

# Load property info from the 'Property & Alias Info.xlsx' file
property_info = pd.read_excel('Property & Alias Info.xlsx', sheet_name='PROPERTY INFO')
property_info = property_info[['PID']]  # Assuming 'PID' is the column for Property ID
property_info.rename(columns={'PID': 'Property ID'}, inplace=True)

# Load alias info from the 'ALIAS INFO' sheet
alias_info = pd.read_excel('Property & Alias Info.xlsx', sheet_name='ALIAS INFO')
 
alias_info.rename(columns={'#': 'Alias ID'}, inplace=True)  # Renaming for clarity

# Step 2: Normalize the columns in each communication data type
# Emails: Relevant columns: 'Property ID', 'Alias ID', 'Date Time', 'Body'
emails = emails[['Property ID', 'Alias ID', 'Date Time', 'Body', 'Attachments']]
emails.rename(columns={'Date Time': 'Date', 'Body': 'Summary of content'}, inplace=True)
emails['Type of communication'] = 'email'

# Text messages: Relevant columns: 'Property ID', 'Alias ID', 'date', 'encrypted_aes_text'
text_messages = text_messages[['Property ID', 'Alias ID', 'date', 'encrypted_aes_text']]
text_messages.rename(columns={'encrypted_aes_text': 'Summary of content', 'date': 'Date'}, inplace=True)
text_messages['Type of communication'] = 'text'

# Voicemails: Relevant columns: 'Property ID', 'Alias ID', 'date', 'transcription_text'
voicemails = voicemails[['Property ID', 'Alias ID', 'date', 'transcription_text']]
voicemails.rename(columns={'transcription_text': 'Summary of content', 'date': 'Date'}, inplace=True)
voicemails['Type of communication'] = 'voicemail'

text_messages['Attachments'] = pd.NA  # Assigning NaN for non-email types
voicemails['Attachments'] = pd.NA

# Step 3: Combine all follow-up data into one DataFrame (Merged Data)
merged_follow_ups = pd.concat([emails, text_messages, voicemails], ignore_index=True)

merged_follow_ups.dropna(subset=['Property ID', 'Alias ID'], inplace=True)

# Add a new column for the primary key (auto-incrementing ID
merged_follow_ups['ID'] = range(1 , len(merged_follow_ups)+ 1 )

merged_follow_ups.to_csv('Merged_Follow_Up_Data.csv', index=False)

# Step 4: Generate a full Cartesian product (many-to-many) of all Property IDs and Alias IDs
property_alias_combinations = pd.merge(
    pd.DataFrame({'Property ID': property_info['Property ID'].unique()}),
    pd.DataFrame({'Alias ID': alias_info['Alias ID'].unique()}),
    how='cross'  # This creates the full combination of each Property ID with each Alias ID
)

# Step 5: Merge the follow-up data with the full property-alias combinations
merged_follow_ups_full = pd.merge(
    property_alias_combinations,
    merged_follow_ups,
    how='left',  # Use left join to keep all property-alias combinations, even if no follow-ups exist
    on=['Property ID', 'Alias ID']
)

# Step 6: Count the number of follow-ups for each property-alias combination
# This filters rows where 'Summary of content' is not null
follow_up_counts = merged_follow_ups_full.groupby(['Property ID', 'Alias ID'])['Summary of content'].count().reset_index(name='Total Follow-Ups')

# Step 7: Ensure combinations with no follow-ups are filled with 'No Follow up'
follow_up_counts['Total Follow-Ups'] = follow_up_counts['Total Follow-Ups'].replace(0, pd.NA).fillna('No Follow up')


# Step 5: Load Titanium Fups and Scandium Fups
titanium_fups = pd.read_excel('Titanium Fups.xlsx')[['Alias ID', 'Property ID']]
scandium_fups = pd.read_excel('Scandium Fups.xlsx')[['Alias ID', 'Property ID']]

# Combine the Fups data and remove duplicates
combined_fups = pd.concat([titanium_fups, scandium_fups]).drop_duplicates()

# Step 6: Use `isin()` to Find Matching Rows
# Create a combined key for comparison
follow_up_counts['combined_key'] = follow_up_counts['Property ID'].astype(str) + '-' + follow_up_counts['Alias ID'].astype(str)
combined_fups['combined_key'] = combined_fups['Property ID'].astype(str) + '-' + combined_fups['Alias ID'].astype(str)

# Set "Not Contacted" where there is no match in the combined Fups data
follow_up_counts['Total Follow-Ups'] = follow_up_counts.apply(
    lambda row: 'Not Contacted' if row['combined_key'] not in combined_fups['combined_key'].values else row['Total Follow-Ups'],
    axis=1
)

# Drop the combined key column
follow_up_counts.drop(columns=['combined_key'], inplace=True)

# Add a new column for the primary key (auto-incrementing ID
follow_up_counts['ID'] = range(1 , len(follow_up_counts)+ 1 )

# Step 8: Save the final follow-up summary report with all property-alias combinations
follow_up_counts.to_csv('Follow_Up_Summary_Report.csv', index=False)

print("Follow-up summary report with many-to-many mapping generated successfully.")


Follow-up summary report with many-to-many mapping generated successfully.


Task 5 

In [3]:
import re

def contains_keywords(content, keywords):
    # Create a regex pattern that combines all keywords, allowing for case-insensitive matches
    pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
    return bool(re.search(pattern, content, re.IGNORECASE))

def classify_follow_up(row, alias_info):
    content = row['Summary of content'].lower() if pd.notna(row['Summary of content']) else ''
    attachments = row['Attachments'] if pd.notna(row['Attachments']) else ''

    # Check for tour confirmation using regex
    tour_confirmation_keywords = [
    'tour confirmation',
    'confirmed your tour',
    'tour is confirmed',
    'tour is booked',
    'appointment tour',
    'appointment is confirmed',
    'tour reservation',
    'tour has been confirmed',
    'confirmation for your tour',
    'confirmed your tour',
    'your appointment'
]
    row['Tour Confirmation'] = 'Yes' if contains_keywords(content, tour_confirmation_keywords) else 'No'

    # Check for Booking Link
    booking_link_keywords = [
    'booking',
    'schedule',
    'book',
    'tour',
    'reservation',
    'reserve'
    ]
    booking_link_pattern = r'(http[s]?://\S+)'  # Regex to identify URLs
    row['Booking Link'] = 'Yes' if contains_keywords(content, booking_link_keywords) and re.search(booking_link_pattern, content) else 'No'

    # Check for Requests Tour Booking
    request_tour_keywords = [
    'booking a tour',
    'schedule a tour',
    'book a tour'
    'reserve a tour',
    'request a tour',
    'arrange a tour'
]
    row['Requests Tour Booking'] = 'Yes' if contains_keywords(content, request_tour_keywords) else 'No'

    # Contains Pictures
    picture_filetypes = ['.jpg', '.jpeg', '.png', '.gif' , '.webp']
    row['Contains Pictures'] = 'Yes' if any(ext in attachments.lower() for ext in picture_filetypes) else 'No'

    # Check for alias name
    if pd.notna(row['Alias ID']):
        alias_row = alias_info.loc[alias_info['Alias ID'] == row['Alias ID']]
        
        if not alias_row.empty:
            alias_name = alias_row['ALIAS NAME'].values[0]
            # Split the alias name into substrings and ignore common words like "or"
            substrings = [word for word in alias_name.lower().split() if word not in ['or']]
            
            # Check if any substring is in the content
            if any(substring in content.lower() for substring in substrings):
                row['Personalized or Generalized'] = 'Personalized'
            else:
                row['Personalized or Generalized'] = 'Generalized'
        else:
            row['Personalized or Generalized'] = 'Generalized'
    else:
        row['Personalized or Generalized'] = 'Generalized'

    return row

# Apply the classification to the follow-up data
classified_follow_ups = merged_follow_ups.apply(lambda row: classify_follow_up(row, alias_info), axis=1)

# Save the classified follow-up data
classified_follow_ups.to_csv('Classified_Follow_Up_Data.csv', index=False)

print("Classified follow-up data has been generated successfully.")


Classified follow-up data has been generated successfully.
